### Talk Time Data Prepare

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# n gram sentence
sentence_check = True

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [6]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [7]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [8]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [9]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/1-Talk Time Data Prepare/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [13]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_200_Word.xlsx", index=False)

In [14]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [ ]:
if sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
else:
    pass


In [15]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [16]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [17]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [19]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...,"[59, 22, 1, 15, 31, 4, 174, 156, 0, 16, 68]"
4,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s,"[4, 61, 14, 49, 117, 22, 97, 4, 121, 0]"
...,...,...,...,...,...,...,...
64,gerçek mi ya ne,1471,1473,gerçek mi ya ne,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,gerçek
65,bence olur bence olmaz,3893,3895,bence olur bence olmaz,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...,bence
66,ben asla öyle bir şey,4768,4769,ben asla öyle bir şey yapmadım,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,asla
67,bir şey yok ki ya adam,2554,2556,bir şey yok ki ya adam hırsızın teki çıktı işte,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...,adam


In [23]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [29]:
len(twogram_set)

334

In [30]:
len(threegram_set)

288

In [31]:
twogram_list = list(twogram_set)
twogram_list

['diye bizi',
 'yok bunun',
 'kadar büyük',
 'ya ne',
 'hemen yine',
 'lütfen ama',
 'şeyi tekrar',
 'için çok',
 'işte uzun',
 'belki böyle',
 'şimdi anne',
 'siz en',
 'kendi şimdi',
 'ol başka',
 'ile biliyorsun',
 'herkes için',
 'demek istiyorsun',
 'şimdi şey',
 'aynı şeyler',
 'fazla olan',
 'bir önce',
 'diye böyle',
 'kadar selam',
 'lütfen ilk',
 'ama sen',
 'ne güzel',
 'biz bunu',
 'ben çok',
 'öyle değil',
 'ya hey',
 'bir de',
 'güzel olsun',
 'önce tek',
 'biraz zaman',
 'gel buraya',
 'olmak bana',
 'kız lazım',
 'bana iyi',
 'bugün biraz',
 'hakkında hiçbir',
 'efendim tekrar',
 'mı diye',
 'iki saat',
 'bir şekilde',
 'başka biri',
 'yani aslında',
 'iki orada',
 'hayır peki',
 'değil misin',
 'biliyorsun sana',
 'musun tamam',
 'yine aynı',
 'birlikte olmaz',
 'gerek yok',
 'zaman öyle',
 'var bir',
 'için gerçekten',
 'tam işte',
 'için artık',
 'olsun efendim',
 'burada ne',
 'çok yok',
 'onları siz',
 'küçük benim',
 'nerede yapıyorsun',
 'olsun tüm',
 'asla öyle'

In [32]:
threegram_list = list(threegram_set)
threegram_list

['önce bir şekilde',
 'var bir de',
 'olabilir belki böyle',
 'söyle bak hala',
 'şimdi geliyor musun',
 'ne güzel çok',
 'gerçek mi ya',
 'bizim gibi nasıl',
 'pekala pekala tamam',
 'bence olur bence',
 'kötü bir şey',
 'gece çok kötü',
 'efendim böyle senin',
 'ben onun hakkında',
 'bakalım burada ne',
 'içinde bizim bütün',
 'için daha doğru',
 'tamam devam et',
 'evet efendim tekrar',
 'bakalım şimdi biz',
 'ile biliyorsun sana',
 'lütfen ilk önce',
 'tekrar seninle birlikte',
 'ilk iş olarak',
 'onlar değil mi',
 'et devam et',
 'bir yardım var',
 'şeyi tekrar tekrar',
 'geliyor mu ya',
 'diye söyle bak',
 'hakkında hiçbir şey',
 'de her zaman',
 'bizim için daha',
 'misin gerek yok',
 'belki bizim için',
 'için size bir',
 'şunu demek istiyorsun',
 'ne oluyor gel',
 'geliyor musun tamam',
 'biri küçük o',
 'asla öyle bir',
 'herkes için de',
 'bana biraz zaman',
 'musun bu şimdi',
 'şimdi anne anne',
 'biliyor biz diye',
 'hayır peki demek',
 'oldu gerçekten harika',
 'ama ya ta

288